In [1]:
import pickle
import os
import networkx as nx
import numpy as np
import pandas as pd
import math
from tqdm import tqdm
import torch
import torch.nn as nn
import pyflagser
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# data loader

In [2]:
import pickle
with open('123 Bus/Dataset_ScaleChange.pkl', 'rb') as g:
    P0Data = pickle.load(g)
N_Senario=len(P0Data)

# generate Label

In [4]:
Class=[]
for i in range(500):
    Class.append(P0Data[i]['Stations Attacked'])
print(Class)

[[4], [6], [], [2], [], [0], [4], [6], [0], [5], [4], [], [4], [5], [3], [4], [0], [5], [6], [0], [6], [2], [5], [1], [0], [], [1], [1], [0], [3], [5], [2], [1], [5], [1], [], [0], [1], [4], [6], [0], [2], [], [1], [6], [4], [4], [6], [0], [0], [6], [], [0], [4], [2], [1], [4], [3], [2], [1], [0], [2], [0], [], [0], [1], [1], [6], [5], [5], [4], [4], [2], [], [0], [4], [3], [], [4], [0], [], [5], [3], [2], [5], [0], [3], [4], [3], [2], [3], [4], [2], [5], [4], [2], [2], [5], [2], [5], [6], [0], [0], [0], [6], [0], [0], [6], [0], [2], [6], [], [0], [4], [3], [2], [3], [], [5], [6], [4], [], [5], [6], [6], [2], [], [1], [5], [1], [1], [3], [5], [3], [0], [1], [], [0], [1], [1], [1], [0], [2], [3], [0], [0], [6], [4], [6], [3], [5], [4], [6], [1], [4], [4], [2], [5], [5], [], [0], [1], [], [3], [3], [], [1], [1], [1], [1], [2], [], [4], [4], [5], [6], [4], [5], [2], [4], [4], [5], [2], [0], [6], [], [6], [1], [4], [6], [0], [], [2], [6], [2], [0], [0], [3], [6], [6], [3], [3], [0], [3], [

In [5]:
def classify_sublist(sublist):
    if sublist == [0]:
        return 0  # Class 0 for []
    elif sublist == [1]:
        return 1  # Class 1 for [0]
    elif sublist == [2]:
        return 2  # Class 2 for [1]
    elif sublist == [3]:
        return 3  # Class 3 for [2]
    elif sublist == [4]:
        return 4  # Class 1 for [0]
    elif sublist == [5]:
        return 5  # Class 2 for [1]
    elif sublist == [6]:
        return 6  # Class 3 for [2]
    else:
        return 7  # For sublists not matching any class (shouldn't happen)

In [6]:
Label=[]
Label = [classify_sublist(sublist) for sublist in Class]

print(Label)

[4, 6, 7, 2, 7, 0, 4, 6, 0, 5, 4, 7, 4, 5, 3, 4, 0, 5, 6, 0, 6, 2, 5, 1, 0, 7, 1, 1, 0, 3, 5, 2, 1, 5, 1, 7, 0, 1, 4, 6, 0, 2, 7, 1, 6, 4, 4, 6, 0, 0, 6, 7, 0, 4, 2, 1, 4, 3, 2, 1, 0, 2, 0, 7, 0, 1, 1, 6, 5, 5, 4, 4, 2, 7, 0, 4, 3, 7, 4, 0, 7, 5, 3, 2, 5, 0, 3, 4, 3, 2, 3, 4, 2, 5, 4, 2, 2, 5, 2, 5, 6, 0, 0, 0, 6, 0, 0, 6, 0, 2, 6, 7, 0, 4, 3, 2, 3, 7, 5, 6, 4, 7, 5, 6, 6, 2, 7, 1, 5, 1, 1, 3, 5, 3, 0, 1, 7, 0, 1, 1, 1, 0, 2, 3, 0, 0, 6, 4, 6, 3, 5, 4, 6, 1, 4, 4, 2, 5, 5, 7, 0, 1, 7, 3, 3, 7, 1, 1, 1, 1, 2, 7, 4, 4, 5, 6, 4, 5, 2, 4, 4, 5, 2, 0, 6, 7, 6, 1, 4, 6, 0, 7, 2, 6, 2, 0, 0, 3, 6, 6, 3, 3, 0, 3, 0, 3, 6, 6, 4, 7, 0, 3, 0, 1, 1, 7, 1, 2, 2, 2, 7, 0, 1, 3, 0, 3, 6, 1, 3, 7, 4, 2, 2, 7, 0, 5, 7, 3, 3, 1, 2, 5, 6, 1, 1, 5, 4, 6, 1, 4, 5, 3, 3, 5, 1, 0, 1, 3, 7, 4, 7, 5, 6, 7, 2, 2, 6, 2, 5, 3, 5, 2, 7, 4, 4, 4, 6, 2, 4, 6, 3, 1, 1, 7, 7, 5, 3, 5, 2, 1, 1, 5, 6, 3, 7, 3, 7, 6, 0, 7, 5, 2, 6, 4, 7, 7, 7, 3, 7, 4, 4, 4, 1, 4, 5, 2, 3, 3, 5, 0, 6, 5, 1, 7, 3, 4, 0, 4, 5, 2, 3, 3, 3, 

# label statistics

In [7]:
from collections import Counter

counter = Counter(Label)

for key, value in counter.items():
    print(f"Element {key} appears {value} times")

Element 4 appears 61 times
Element 6 appears 65 times
Element 7 appears 68 times
Element 2 appears 59 times
Element 0 appears 64 times
Element 5 appears 58 times
Element 3 appears 61 times
Element 1 appears 64 times


# make the node voltage as time series format

In [30]:
def Average(lst):
    return sum(lst) / len(lst)
def make_timeseries(P0Data,scenario, total_time_step,Nodes,Edges):
    time_series_voltage=[]
    time_series_BFlow=[]
    for time_step in range(total_time_step):
        voltage=[]
        B_flow=[]
        for node in Nodes:
            node_voltage=P0Data[scenario]['Bus Voltages'][node][time_step]
            #voltage.append(P0Data[scenario]['Bus Voltages'][node][time_step].tolist())
            voltage.append(Average(node_voltage))
        time_series_voltage.append(voltage)
        for edge in Edges:
            B_flow.append([P0Data[scenario]['Branch flows'][edge][time_step]])
        time_series_BFlow.append(B_flow)
    return time_series_voltage, time_series_BFlow

# construct graph G

In [10]:
node = P0Data[0]['Bus Voltages'].keys()
Edge = P0Data[0]['Branch flows'].keys()
# Initialize an empty graph
G = nx.Graph()  # or nx.DiGraph() for a directed graph

# Add nodes (convert dict_keys to a list if needed)
G.add_nodes_from(list(node))

# Add edges
G.add_edges_from(Edge)

In [11]:
N_Scenario=500
N=list(G.nodes)
E=list(G.edges)

# construct mean node voltage (only baseline)

In [31]:
N_Scenario=500
N=list(G.nodes)
E=list(G.edges)
feat = [make_timeseries(P0Data,i, 288,N,E)[0] for i in tqdm(range(N_Scenario), desc="Processing")]


Processing:  39%|███████████▏                 | 193/500 [06:36<15:42,  3.07s/it]


Processing:  79%|██████████████████████▊      | 394/500 [13:00<01:45,  1.01it/s]


Processing: 100%|█████████████████████████████| 500/500 [16:43<00:00,  2.01s/it]


In [13]:
from tqdm import tqdm

# extract topological features

In [43]:
def print_stat(train_acc, test_acc,pre,rec,f1):
    argmax=np.argmax(train_acc)
    best_result=test_acc[argmax]
    train_acc=np.max(train_acc)
    test_acc=np.max(test_acc)
    print(f'Train accuracy = {train_acc*100:.2f}%,Test Accuracy = {test_acc*100:.2f}%,precision = {np.max(pre)*100:.2f}%,Recall = {np.max(rec)*100:.2f}%,f1-score = {np.max(f1)*100:.2f}%\n')
    return test_acc

def Average(lst):
    return sum(lst) / len(lst)
def make_timeseries(scenario, total_time_step):
    time_series_voltage=[]
    time_series_BFlow=[]
    for time_step in range(total_time_step):
        voltage=[]
        B_flow=[]
        for node in N:
            node_voltage=P0Data[scenario]['Bus Voltages'][node][time_step]
            #voltage.append(P0Data[scenario]['Bus Voltages'][node][time_step].tolist())
            voltage.append(Average(node_voltage))
        time_series_voltage.append(voltage)
        for edge in E:
            B_flow.append([P0Data[scenario]['Branch flows'][edge][time_step]])
        time_series_BFlow.append(B_flow)
    return time_series_voltage, time_series_BFlow

def Topo_Fe_TimeSeries_MP(TS_voltage, TS_branchFlow, F_voltage,F_Flow):
    betti_0=[]
    for k in range(len(TS_voltage)):
        fec=[]
        AverageVoltage=[]
        Voltage=TS_voltage[k]
        for y in Voltage:
            AverageVoltage.append(y)
        #AverageVoltage = [i * 100 for i in AverageVoltage]
        BranchFlow=[]
        Branch_Flow=TS_branchFlow[k]
        for j  in range(len(Branch_Flow)):
            BranchFlow.append(Branch_Flow[j][0])

        for p in range(len(F_voltage)):
            Active_node_v=np.where(np.array(AverageVoltage) > F_voltage[p])[0].tolist()
            for q in range(len(F_Flow)):
                #if AverageVoltage[p]> F_voltage[p] and BranchFlow[q]> F_Flow[q]:
                #n_active = np.where(np.array(AverageVoltage) > F_voltage[p])[0].tolist()
                n_active=Active_node_v.copy()
                #print(n_active)
                G = nx.DiGraph()
                G.add_nodes_from(n_active)
                indices = np.where(np.array(BranchFlow) > F_Flow[q])[0].tolist()
                for s in indices:
                    a=int(N.index(E[s][0]))
                    b=int(N.index(E[s][1]))
                    if a in n_active and b in n_active:
                        G.add_edge(a, b)
                    #n_active.append(int(N.index(E[s][0])))
                    #n_active.append(int(N.index(E[s][1])))
                #Active_node=np.unique(n_active)
                #print(G.edges())
                if (len(n_active)==0):
                    fec.append(0)
                else:
                    #b=A[Active_node,:][:,Active_node]
                    Adj = nx.to_numpy_array(G)
                    my_flag=pyflagser.flagser_unweighted(Adj, min_dimension=0, max_dimension=2, directed=False, coeff=2, approximation=None)
                    x = my_flag["betti"]
                    fec.append(x[0])
                n_active.clear()
            Active_node_v.clear()
        betti_0.append(fec)
    return betti_0


In [16]:
# voltage_threshold = np.array([0.34, 0.35, 0.94, 0.98, 0.99, 1.01, 1.02, 1.03, 1.04, 1.05])

# BFlow_threshold = np.array([8.8e-01, 1.75e+00, 7.24e+00, 2.453e+01, 5.928e+01,
#                             1.3625e+02, 3.9623e+02, 5.0359e+02, 1.29343e+03,
#                             1.33055e+03, 1.60132e+03, 1.66341e+03, 1.73374e+03])
voltage_threshold=np.array([1.04, 1.03, 1.02,1.0, 0.99, 0.98, 0.96,0.95,0.94,0.93,0.91, 0.35, 0.34])
# BFlow_threshold=np.array([1.73374e+03, 1.66341e+03, 1.33055e+03, 
#                                 5.03590e+02, 1.36250e+02, 5.92800e+01, 2.45300e+01, 
#                                 7.24000e+00, 8.80000e-01])

BFlow_threshold=np.array([1.73374e+03, 1.33055e+03, 
                                5.03590e+02, 1.36250e+02, 5.92800e+01, 2.45300e+01, 
                                7.24000e+00, 8.80000e-01])
def extract_topological_features(graph_id):
    time_series=make_timeseries(graph_id,time_step)
    betti=Topo_Fe_TimeSeries_MP(time_series[0],time_series[1], voltage_threshold,BFlow_threshold)
    return betti

def process_graph(graph_id):
    # Function to extract topological features from a single graph
    return extract_topological_features(graph_id)

In [19]:
time_step=288
feat_mp_500=[]
for i in tqdm(range(200,500),desc='processing'):
    bet=process_graph(i)
    feat_mp_500.append(bet)


processing: 100%|███████████████████████████| 300/300 [1:32:05<00:00, 18.42s/it]


# save the topological features

In [20]:
with open('EV_123bus_betti0_500_scaled.data', 'wb') as f:
    pickle.dump(feat_mp_500, f)

# load the topological features

In [46]:
#first 200 scenarios
with open('EV_123bus_betti0_200_scaled.data', 'rb') as f:
        mp_data = pickle.load(f)

In [47]:
#next 300 scenarios
with open('EV_123bus_betti0_500_scaled.data', 'rb') as f:
        mp_data300 = pickle.load(f)

In [48]:
#combine all topological features
data_mp=mp_data+mp_data300

In [49]:

from sklearn.preprocessing import MinMaxScaler
X0=np.array(data_mp)
y=Label

# X=[]
# for i in range(N_Senario):

#     scaler = MinMaxScaler()

# # Fit scaler to data and transform data
#     X.append(scaler.fit_transform(X0[i]))

# Convert data to PyTorch tensors
X = torch.tensor(X0, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

num_samples = N_Senario
num_timesteps = 288
#num_features = len(F_voltage)*len(F_Flow)
num_features = len(X[0][0])
num_classes = len(np.unique(y))

In [38]:
print(num_classes)

8


In [50]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score


# Define the Transformer model with Batch Normalization
class TransformerClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_heads, n_layers, num_timesteps):
        super(TransformerClassifier, self).__init__()
        self.embedding = nn.Linear(input_dim, hidden_dim)
        self.positional_encoding = nn.Parameter(torch.zeros(1, num_timesteps, hidden_dim))
        self.transformer = nn.Transformer(d_model=hidden_dim, nhead=n_heads, num_encoder_layers=n_layers, num_decoder_layers=n_layers)
        self.fc = nn.Linear(hidden_dim * num_timesteps, output_dim)  # Flatten the output of the transformer
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, src):
        src_emb = self.embedding(src) + self.positional_encoding[:, :src.size(1), :]
        src_emb = src_emb.permute(1, 0, 2)  # (seq_len, batch, feature)
        transformer_output = self.transformer.encoder(src_emb)
        transformer_output = transformer_output.permute(1, 0, 2).contiguous().view(src.size(0), -1)  # Flatten
        transformer_output = self.dropout(transformer_output)
        predictions = self.fc(transformer_output)
        return predictions


def reset_weights(model):
    for layer in model.children():
        if hasattr(layer, 'reset_parameters'):
            layer.reset_parameters()

# Define input and output dimensions (example placeholders)
input_dim = num_features
hidden_dim = 64
output_dim = num_classes
n_heads = 2
n_layers = 2
num_timesteps = 288  # Adjust based on your sequence length

# Initialize model, loss function, and optimizer
model = TransformerClassifier(input_dim, hidden_dim, output_dim, n_heads, n_layers, num_timesteps)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=23)

# Create DataLoader
train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Lists to store metrics
train_losses = []
train_accuracies = []
test_accuracies = []
precisions = []
recalls = []
f1_scores = []

# Train the model
reset_weights(model)
model.train()
for epoch in range(600):
    epoch_train_loss = 0
    correct_train = 0
    total_train = 0
    
    # Training loop
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()

        # Track training loss and accuracy
        epoch_train_loss += loss.item()
        _, predicted = torch.max(output, 1)
        total_train += y_batch.size(0)
        correct_train += (predicted == y_batch).sum().item()

    avg_train_loss = epoch_train_loss / len(train_loader)
    train_accuracy = correct_train / total_train
    train_losses.append(avg_train_loss)
    train_accuracies.append(train_accuracy)

    # Evaluate on the test set
    model.eval()
    correct_test = 0
    total_test = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            output = model(X_batch)
            _, predicted = torch.max(output, 1)
            total_test += y_batch.size(0)
            correct_test += (predicted == y_batch).sum().item()

            # Store predictions and targets for metrics
            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(y_batch.cpu().numpy())

    test_accuracy = correct_test / total_test
    test_accuracies.append(test_accuracy)
    
    # Calculate precision, recall, and F1-score
    precision = precision_score(all_targets, all_preds, average='weighted',zero_division=0)
    recall = recall_score(all_targets, all_preds, average='weighted',zero_division=0)
    f1 = f1_score(all_targets, all_preds, average='weighted',zero_division=0)

    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

    model.train()  # Switch back to training mode

    # Print metrics for this epoch
    print(f'Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f}, Train Accuracy = {train_accuracy:.4f}%, Test Accuracy = {test_accuracy:.4f}%')
    print(f'Precision = {precision:.2f}, Recall = {recall:.4f}, F1-Score = {f1:.4f}')
print('=============\n finished training\n=============')
acc=print_stat(train_accuracies, test_accuracies,precisions,recalls,f1_scores)


/Users/joshem/anaconda3/envs/tensorflow/lib/python3.10/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 1: Train Loss = 8.9065, Train Accuracy = 0.1400%, Test Accuracy = 0.1800%
Precision = 0.03, Recall = 0.1800, F1-Score = 0.0549
Epoch 2: Train Loss = 4.5875, Train Accuracy = 0.1250%, Test Accuracy = 0.1500%
Precision = 0.02, Recall = 0.1500, F1-Score = 0.0391
Epoch 3: Train Loss = 3.2166, Train Accuracy = 0.1225%, Test Accuracy = 0.1000%
Precision = 0.01, Recall = 0.1000, F1-Score = 0.0182
Epoch 4: Train Loss = 2.7557, Train Accuracy = 0.1475%, Test Accuracy = 0.1000%
Precision = 0.01, Recall = 0.1000, F1-Score = 0.0182
Epoch 5: Train Loss = 2.6174, Train Accuracy = 0.0950%, Test Accuracy = 0.0300%
Precision = 0.00, Recall = 0.0300, F1-Score = 0.0017
Epoch 6: Train Loss = 2.8265, Train Accuracy = 0.0950%, Test Accuracy = 0.1500%
Precision = 0.02, Recall = 0.1500, F1-Score = 0.0391
Epoch 7: Train Loss = 2.8656, Train Accuracy = 0.1175%, Test Accuracy = 0.1300%
Precision = 0.02, Recall = 0.1300, F1-Score = 0.0299
Epoch 8: Train Loss = 2.7793, Train Accuracy = 0.1175%, Test Accuracy

Epoch 63: Train Loss = 1.5259, Train Accuracy = 0.4025%, Test Accuracy = 0.3300%
Precision = 0.38, Recall = 0.3300, F1-Score = 0.2767
Epoch 64: Train Loss = 1.5460, Train Accuracy = 0.4300%, Test Accuracy = 0.3200%
Precision = 0.35, Recall = 0.3200, F1-Score = 0.2676
Epoch 65: Train Loss = 1.5346, Train Accuracy = 0.4175%, Test Accuracy = 0.3100%
Precision = 0.33, Recall = 0.3100, F1-Score = 0.2570
Epoch 66: Train Loss = 1.4773, Train Accuracy = 0.4350%, Test Accuracy = 0.3300%
Precision = 0.38, Recall = 0.3300, F1-Score = 0.2772
Epoch 67: Train Loss = 1.4475, Train Accuracy = 0.4050%, Test Accuracy = 0.3200%
Precision = 0.38, Recall = 0.3200, F1-Score = 0.2602
Epoch 68: Train Loss = 1.4166, Train Accuracy = 0.4525%, Test Accuracy = 0.3300%
Precision = 0.38, Recall = 0.3300, F1-Score = 0.2817
Epoch 69: Train Loss = 1.4748, Train Accuracy = 0.4225%, Test Accuracy = 0.4300%
Precision = 0.52, Recall = 0.4300, F1-Score = 0.3997
Epoch 70: Train Loss = 1.3173, Train Accuracy = 0.4850%, Test 

Epoch 124: Train Loss = 0.5895, Train Accuracy = 0.7525%, Test Accuracy = 0.5100%
Precision = 0.50, Recall = 0.5100, F1-Score = 0.4457
Epoch 125: Train Loss = 0.7019, Train Accuracy = 0.7275%, Test Accuracy = 0.3800%
Precision = 0.46, Recall = 0.3800, F1-Score = 0.3444
Epoch 126: Train Loss = 0.7672, Train Accuracy = 0.7050%, Test Accuracy = 0.4500%
Precision = 0.43, Recall = 0.4500, F1-Score = 0.3874
Epoch 127: Train Loss = 0.6768, Train Accuracy = 0.7250%, Test Accuracy = 0.4500%
Precision = 0.45, Recall = 0.4500, F1-Score = 0.3830
Epoch 128: Train Loss = 0.6706, Train Accuracy = 0.6850%, Test Accuracy = 0.5300%
Precision = 0.66, Recall = 0.5300, F1-Score = 0.4980
Epoch 129: Train Loss = 0.6753, Train Accuracy = 0.7100%, Test Accuracy = 0.5100%
Precision = 0.50, Recall = 0.5100, F1-Score = 0.4470
Epoch 130: Train Loss = 0.5570, Train Accuracy = 0.7525%, Test Accuracy = 0.3800%
Precision = 0.32, Recall = 0.3800, F1-Score = 0.2814
Epoch 131: Train Loss = 0.6429, Train Accuracy = 0.7325

Epoch 185: Train Loss = 0.3922, Train Accuracy = 0.8175%, Test Accuracy = 0.6100%
Precision = 0.55, Recall = 0.6100, F1-Score = 0.5501
Epoch 186: Train Loss = 0.4380, Train Accuracy = 0.7850%, Test Accuracy = 0.6100%
Precision = 0.56, Recall = 0.6100, F1-Score = 0.5507
Epoch 187: Train Loss = 0.3832, Train Accuracy = 0.8275%, Test Accuracy = 0.6100%
Precision = 0.68, Recall = 0.6100, F1-Score = 0.5976
Epoch 188: Train Loss = 0.4070, Train Accuracy = 0.8175%, Test Accuracy = 0.5300%
Precision = 0.53, Recall = 0.5300, F1-Score = 0.4826
Epoch 189: Train Loss = 0.3624, Train Accuracy = 0.8225%, Test Accuracy = 0.5100%
Precision = 0.52, Recall = 0.5100, F1-Score = 0.4633
Epoch 190: Train Loss = 0.4517, Train Accuracy = 0.8075%, Test Accuracy = 0.5000%
Precision = 0.51, Recall = 0.5000, F1-Score = 0.4518
Epoch 191: Train Loss = 0.5295, Train Accuracy = 0.7975%, Test Accuracy = 0.4500%
Precision = 0.47, Recall = 0.4500, F1-Score = 0.3832
Epoch 192: Train Loss = 0.6625, Train Accuracy = 0.7250

...

Epoch 307: Train Loss = 0.3443, Train Accuracy = 0.8550%, Test Accuracy = 0.6900%
Precision = 0.82, Recall = 0.6900, F1-Score = 0.6629
Epoch 308: Train Loss = 0.3466, Train Accuracy = 0.8500%, Test Accuracy = 0.7200%
Precision = 0.79, Recall = 0.7200, F1-Score = 0.7114
Epoch 309: Train Loss = 0.3502, Train Accuracy = 0.8325%, Test Accuracy = 0.7600%
Precision = 0.81, Recall = 0.7600, F1-Score = 0.7376
Epoch 310: Train Loss = 0.3153, Train Accuracy = 0.8625%, Test Accuracy = 0.7400%
Precision = 0.84, Recall = 0.7400, F1-Score = 0.7155
Epoch 311: Train Loss = 0.3981, Train Accuracy = 0.8425%, Test Accuracy = 0.7000%
Precision = 0.66, Recall = 0.7000, F1-Score = 0.6522
Epoch 312: Train Loss = 0.4710, Train Accuracy = 0.8200%, Test Accuracy = 0.7200%
Precision = 0.82, Recall = 0.7200, F1-Score = 0.6778
Epoch 313: Train Loss = 0.3360, Train Accuracy = 0.8900%, Test Accuracy = 0.7600%
Precision = 0.78, Recall = 0.7600, F1-Score = 0.7264
Epoch 314: Train Loss = 0.3785, Train Accuracy = 0.8225

Epoch 368: Train Loss = 0.2610, Train Accuracy = 0.8950%, Test Accuracy = 0.7300%
Precision = 0.68, Recall = 0.7300, F1-Score = 0.6805
Epoch 369: Train Loss = 0.3068, Train Accuracy = 0.8825%, Test Accuracy = 0.7200%
Precision = 0.76, Recall = 0.7200, F1-Score = 0.6971
Epoch 370: Train Loss = 0.2350, Train Accuracy = 0.9025%, Test Accuracy = 0.8100%
Precision = 0.83, Recall = 0.8100, F1-Score = 0.8056
Epoch 371: Train Loss = 0.2490, Train Accuracy = 0.9025%, Test Accuracy = 0.6900%
Precision = 0.66, Recall = 0.6900, F1-Score = 0.6472
Epoch 372: Train Loss = 0.2775, Train Accuracy = 0.8875%, Test Accuracy = 0.7500%
Precision = 0.81, Recall = 0.7500, F1-Score = 0.7309
Epoch 373: Train Loss = 0.2432, Train Accuracy = 0.8975%, Test Accuracy = 0.7600%
Precision = 0.83, Recall = 0.7600, F1-Score = 0.7443
Epoch 374: Train Loss = 0.2340, Train Accuracy = 0.8875%, Test Accuracy = 0.7600%
Precision = 0.77, Recall = 0.7600, F1-Score = 0.7429
Epoch 375: Train Loss = 0.2219, Train Accuracy = 0.9025

Epoch 429: Train Loss = 0.3419, Train Accuracy = 0.8575%, Test Accuracy = 0.7500%
Precision = 0.83, Recall = 0.7500, F1-Score = 0.7358
Epoch 430: Train Loss = 0.2540, Train Accuracy = 0.9050%, Test Accuracy = 0.8700%
Precision = 0.91, Recall = 0.8700, F1-Score = 0.8721
Epoch 431: Train Loss = 0.2818, Train Accuracy = 0.8950%, Test Accuracy = 0.8500%
Precision = 0.90, Recall = 0.8500, F1-Score = 0.8536
Epoch 432: Train Loss = 0.1369, Train Accuracy = 0.9450%, Test Accuracy = 0.9000%
Precision = 0.92, Recall = 0.9000, F1-Score = 0.9031
Epoch 433: Train Loss = 0.1498, Train Accuracy = 0.9425%, Test Accuracy = 0.8600%
Precision = 0.92, Recall = 0.8600, F1-Score = 0.8637
Epoch 434: Train Loss = 0.1125, Train Accuracy = 0.9575%, Test Accuracy = 0.9100%
Precision = 0.93, Recall = 0.9100, F1-Score = 0.9113
Epoch 435: Train Loss = 0.1694, Train Accuracy = 0.9350%, Test Accuracy = 0.8500%
Precision = 0.90, Recall = 0.8500, F1-Score = 0.8510
Epoch 436: Train Loss = 0.1151, Train Accuracy = 0.9625

Epoch 490: Train Loss = 0.0426, Train Accuracy = 0.9850%, Test Accuracy = 0.9400%
Precision = 0.95, Recall = 0.9400, F1-Score = 0.9417
Epoch 491: Train Loss = 0.0380, Train Accuracy = 0.9825%, Test Accuracy = 0.9000%
Precision = 0.93, Recall = 0.9000, F1-Score = 0.9039
Epoch 492: Train Loss = 0.0444, Train Accuracy = 0.9900%, Test Accuracy = 0.9000%
Precision = 0.93, Recall = 0.9000, F1-Score = 0.9039
Epoch 493: Train Loss = 0.0410, Train Accuracy = 0.9850%, Test Accuracy = 0.9700%
Precision = 0.97, Recall = 0.9700, F1-Score = 0.9705
Epoch 494: Train Loss = 0.0799, Train Accuracy = 0.9700%, Test Accuracy = 0.9300%
Precision = 0.94, Recall = 0.9300, F1-Score = 0.9320
Epoch 495: Train Loss = 0.0818, Train Accuracy = 0.9700%, Test Accuracy = 0.8500%
Precision = 0.90, Recall = 0.8500, F1-Score = 0.8548
Epoch 496: Train Loss = 0.0374, Train Accuracy = 0.9850%, Test Accuracy = 0.9000%
Precision = 0.92, Recall = 0.9000, F1-Score = 0.9028
Epoch 497: Train Loss = 0.0633, Train Accuracy = 0.9750

Epoch 551: Train Loss = 0.5368, Train Accuracy = 0.8025%, Test Accuracy = 0.7900%
Precision = 0.84, Recall = 0.7900, F1-Score = 0.7922
Epoch 552: Train Loss = 0.7227, Train Accuracy = 0.7400%, Test Accuracy = 0.8200%
Precision = 0.87, Recall = 0.8200, F1-Score = 0.8254
Epoch 553: Train Loss = 0.6666, Train Accuracy = 0.7425%, Test Accuracy = 0.8400%
Precision = 0.87, Recall = 0.8400, F1-Score = 0.8426
Epoch 554: Train Loss = 0.6084, Train Accuracy = 0.7875%, Test Accuracy = 0.8700%
Precision = 0.88, Recall = 0.8700, F1-Score = 0.8691
Epoch 555: Train Loss = 0.5611, Train Accuracy = 0.8175%, Test Accuracy = 0.8800%
Precision = 0.91, Recall = 0.8800, F1-Score = 0.8887
Epoch 556: Train Loss = 0.6194, Train Accuracy = 0.7850%, Test Accuracy = 0.9000%
Precision = 0.91, Recall = 0.9000, F1-Score = 0.8998
Epoch 557: Train Loss = 0.4998, Train Accuracy = 0.8150%, Test Accuracy = 0.9200%
Precision = 0.94, Recall = 0.9200, F1-Score = 0.9253
Epoch 558: Train Loss = 0.4393, Train Accuracy = 0.8550